In [2]:
import pandas as pd
import pickle
import seaborn as sns
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, accuracy_score, plot_roc_curve, plot_confusion_matrix, roc_curve, confusion_matrix
import itertools
from tensorflow.keras.initializers import Constant, TruncatedNormal
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from numpy import mean, absolute

07_optimizeFairnessallT

-> -> für jede Variable wurden die Daten ausbalanciert -> ausbalanciert nach y und nach der jeweiligen variable BEVOR der Datensatz in T matrizen zerteilt wird

Balance Out Dataset

In [3]:
infile = open('../01_prognosemodell/06_newtry/fairness_ready.pkl','rb')
df = pickle.load(infile)
infile.close()

In [4]:
df_grouped = df.groupby('UebungsID').agg({'AbiEltern':'first','ID':'count','Erstloesung': 'sum', 'Schussel': 'sum', 'Erfolg': 'sum', 'Schwierigkeit': 'mean', 'ist_Schulzeit': 'first', 'MehrfachFalsch':'first', 'vorher_abgebrochen':'first', 'y':'first','UserAttribut':'first',
'Klassenstufe':'first', 'Jahredabei':'first', 'AnzahlAufgaben':'first', 'Art__GK':'first', 'Art__GR':'first', 'Art__GZ':'first',
       'Art__K':'first', 'Art__LB':'first', 'HA__HA':'first', 'HA__Self':'first', 'HA__nt':'first', 'HA__vt':'first', 'HA__zt':'first',
       'Sex__m':'first', 'Sex__w':'first', 'Testposition__pruefung':'sum', 'Testposition__training':'sum','Testposition__version':'sum', 'OrderNumber':'max', 'steps':'max'})

df_grouped['Fehler'] = df_grouped['ID'] - df_grouped['Erfolg']
df_grouped = df_grouped.reset_index()

In [5]:
#balance out dataset y
df_grouped_1 = df_grouped[df_grouped.y == 1]
df_grouped_0 = df_grouped[df_grouped.y == 0]

print(len(df_grouped_0))
print(len(df_grouped_1))

df_grouped_1 = df_grouped_1.sample(n=7528)
df_grouped = df_grouped_1.append(df_grouped_0)
ids = df_grouped[['UebungsID']]
df = pd.merge(df, ids, on='UebungsID')

7528
40578


C:\Users\Nathalie\AppData\Local\Temp\ipykernel_20484\1688392349.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grouped = df_grouped_1.append(df_grouped_0)


In [6]:
#balance out dataset AbiEltern
df.AbiEltern = df.AbiEltern.astype('float')
df_grouped_1 = df[df.AbiEltern> 0]
df_grouped_0 = df[df.AbiEltern == 0]

# print(len(df_grouped_0))
# print(len(df_grouped_1))

df_grouped_1 = df_grouped_1.sample(n=2446)
df_grouped = df_grouped_1.append(df_grouped_0)
df = df_grouped

C:\Users\Nathalie\AppData\Local\Temp\ipykernel_20484\2884032262.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_grouped = df_grouped_1.append(df_grouped_0)


In [7]:
df_1 = df.loc[df['OrderNumber'] ==1]
df_1.to_pickle('AbiEltern_allsessions/matrix1.pkl')

n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]

for x in n:
    df_x = df.loc[df['OrderNumber'] < (x+1)]
    df_x = df_x.groupby('UebungsID').agg({'ID':'count','Erstloesung': 'sum', 'Schussel': 'sum', 'Erfolg': 'sum', 'Schwierigkeit': 'mean', 'ist_Schulzeit': 'first', 'MehrfachFalsch':'first', 'vorher_abgebrochen':'first', 'y':'first','UserAttribut':'first', 'Fehler':'first', 
'Klassenstufe':'first', 'Jahredabei':'first', 'AnzahlAufgaben':'first', 'Art__GK':'first', 'Art__GR':'first', 'Art__GZ':'first',
       'Art__K':'first', 'Art__LB':'first', 'HA__HA':'first', 'HA__Self':'first', 'HA__nt':'first', 'HA__vt':'first', 'HA__zt':'first',
       'Sex__m':'first', 'Sex__w':'first', 'Testposition__pruefung':'sum', 'Testposition__training':'sum','Testposition__version':'sum', 'OrderNumber':'max', 'steps':'max'})

    df_x = df_x.loc[df_x['ID'] ==x]     
    path = 'AbiEltern_allsessions/matrix'+ str(x) +'.pkl'
    df_x.to_pickle(path)

Models

In [8]:
metrics = pd.DataFrame(columns=['model', 'group', 'subgroup', 'Sentence', 'Accuracy', 'Precision', 'Recall', 'AUC', 'FPR'])

In [9]:
def get_metrics(clf,X,y,cv,pred):
    a = accuracy_score(y,pred)
    p = precision_score(y,pred)
    r = recall_score(y,pred)
    roc_auc = roc_auc_score(y,pred)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    fpr = fp/(fp+tn)

    return a,p,r,roc_auc,fpr

In [ ]:
n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    #build models
    path='AbiEltern_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    X = df[feature_cols]
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    k = 5
    cv = KFold(n_splits=k, random_state=None)

    #DTE
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=5)
    clf = clf.fit(X_train,y_train)
    pred = clf.predict(X_test)

    a,p,r,roc_auc,fpr = get_metrics(clf,X_test,y_test,cv,pred)

    metrics = metrics.append({'model':'DTE','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern']
    subgroup = ['abi', 'keinAbi']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi']

    for (group, subgroup, matrix) in zip(group, subgroup, matrice):
        path= '../01_prognosemodell/06_newtry/'+matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        X = df[feature_cols]
        y = df.y
        y= y.astype('int')
        pred = clf.predict(X)
        
        a,p,r,roc_auc,fpr = get_metrics(clf,X,y,cv,pred)
        metrics = metrics.append({'model':'DTE','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

In [ ]:
n =  [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    #build models
    path='AbiEltern_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()
    X = df[feature_cols]
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    k = 5
    cv = KFold(n_splits=k, random_state=None)

    #knn
    knn = KNeighborsClassifier(n_neighbors=2)
    knn = clf.fit(X_train, y_train)

    pred = clf.predict(X_test)
    a,p,r,roc_auc,fpr = get_metrics(knn,X_test,y_test,cv,pred)
    metrics = metrics.append({'model':'KNN','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern']
    subgroup = ['abi', 'keinAbi']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi']
    
    for (group, subgroup, matrix) in zip(group, subgroup, matrice):
        path= '../01_prognosemodell/06_newtry/'+matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        X = df[feature_cols]
        y = df.y
        y= y.astype('int')
        pred = clf.predict(X)

        a,p,r,roc_auc,fpr = get_metrics(clf,X,y,cv,pred)
        metrics = metrics.append({'model':'KNN','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

In [ ]:
# baseline model
def build_model():
	# create model
	model = Sequential()
	model.add(Dense(24, input_dim=24, activation='relu'))
	model.add(Dense(48, activation='relu'))
	model.add(Dense(24, activation='relu'))
	model.add(Dense(12, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
		
	return model


def get_dn_metrics(model, X,y):
    yhat_probs = model.predict(X, verbose=0)
    yhat_classes =  (model.predict(X) > 0.5).astype("int32")
    # reduce to 1d array
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]
    a = accuracy_score(y, yhat_classes)
    p = precision_score(y, yhat_classes)
    r = recall_score(y, yhat_classes)
    roc_auc = roc_auc_score(y, yhat_probs)
    tn, fp, fn, tp = confusion_matrix(y, yhat_classes).ravel()
    fpr = fp/(fp+tn)

    return a,p,r,roc_auc,fpr

n =  [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]
feature_cols = ['Erstloesung','Schussel','Erfolg', 'Schwierigkeit', 'ist_Schulzeit', 'MehrfachFalsch', 'vorher_abgebrochen','Fehler', 'Klassenstufe', 'Jahredabei', 'AnzahlAufgaben', 'Sex__m', 'Sex__w', 'Testposition__pruefung', 'Testposition__training','Testposition__version', 'Art__GK', 'Art__GR', 'Art__GZ', 'Art__K', 'Art__LB','UserAttribut', 'OrderNumber', 'steps']

for i in n:
    path='AbiEltern_allsessions/matrix'+str(i)+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()
    df=df.reset_index()

    y_len = len(feature_cols)
    X = df[feature_cols].astype(float)
    y = df.y
    y= y.astype('int')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

    model = build_model()

    model.compile(
        loss="binary_crossentropy",
        optimizer='Adam',
        metrics=["accuracy"]
    )

    model.fit(
        x=X_train,
        y=y_train,
        epochs=10,
        batch_size=128,
        verbose=0,
        validation_data=(X_test, y_test)
    )

    scores = model.evaluate(
        x=X_test,
        y=y_test,
        verbose=0
    )

    a,p,r,roc_auc,fpr= get_dn_metrics(model, X_test,y_test)
    metrics = metrics.append({'model':'DL','group':'all','subgroup':'all','Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

    group = ['abiEltern', 'abiEltern']
    subgroup = ['abi', 'keinAbi']
    matrice = ['matrices_forte_abi', 'matrices_forte_keinAbi']

    for (group, subgroup, matrice) in zip(group, subgroup, matrice):
        path= '../01_prognosemodell/06_newtry/'+matrix+'/matrix'+str(i)+'.pkl'
        infile = open(path,'rb')
        df = pickle.load(infile)
        infile.close()
        df=df.reset_index()
        y_len = len(feature_cols)
        X = df[feature_cols].astype(float)
        y = df.y
        y= y.astype('int')

        a,p,r,roc_auc,fpr= get_dn_metrics(model, X,y)

        metrics = metrics.append({'model':'DL','group':group,'subgroup':subgroup,'Sentence': i, 'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

In [13]:
metrics

,model,group,subgroup,Sentence,Accuracy,Precision,Recall,AUC,FPR
0,DTE,all,all,2,0.760522,0.726891,0.908136,0.743029,0.422078
1,DTE,abiEltern,abi,2,0.873658,0.931882,0.92338,0.7218,0.479781
2,DTE,abiEltern,keinAbi,2,0.857463,0.936859,0.897396,0.738925,0.419546
3,DTE,all,all,3,0.767267,0.751693,0.880952,0.749504,0.381944
4,DTE,abiEltern,abi,3,0.859366,0.944673,0.893862,0.740752,0.412359
...,...,...,...,...,...,...,...,...,...
526,DL,abiEltern,abi,59,0.944444,0.944444,1.0,0.221789,1.0
527,DL,abiEltern,keinAbi,59,0.944444,0.944444,1.0,0.221789,1.0
528,DL,all,all,60,0.666667,0.769231,0.769231,0.576923,0.6
529,DL,abiEltern,abi,60,0.607759,0.963768,0.607306,0.659466,0.384615


Evaluate

In [14]:
df = metrics
grouped = df.groupby(df.group)
df_abiEltern = grouped.get_group("abiEltern")

df_abiEltern = df_abiEltern.drop(columns=['group', 'Accuracy'])
df_abiEltern = pd.pivot_table(df_abiEltern, values=["Precision","Recall","AUC","FPR"], index=["model", "Sentence"], columns=["subgroup"])
df_abiEltern['PP'] = df_abiEltern.Precision.abi-df_abiEltern.Precision.keinAbi
df_abiEltern['EO'] = df_abiEltern.Recall.keinAbi-df_abiEltern.Recall.abi
df_abiEltern['SA'] = df_abiEltern.AUC.abi-df_abiEltern.AUC.keinAbi
df_abiEltern['PE'] = df_abiEltern.FPR.keinAbi-df_abiEltern.FPR.abi
df_abiEltern = df_abiEltern.drop(columns=['AUC','Precision','Recall','FPR'])
df_abiEltern.columns = df_abiEltern.columns.droplevel(1)
df_abiEltern = pd.pivot_table(df_abiEltern, values=["PP","EO","SA","PE"], index=["Sentence"], columns=["model"])



In [15]:
#format result

def threshold001(v, props=''):
    return props if (v > 0.02) or (v < -0.02) else None

def threshold005(v, props=''):
    return props if (v > 0.05) or (v < -0.05) else None

def negativeValue(v, props=''):
    return props if (v < 0) else None

def showTable(df):
    styled = df.style.set_properties(color="black", align="right")\
        .set_properties(**{'background-color': 'white'})\
        .applymap(threshold001, props='color:orange;')\
        .applymap(threshold005, props='color:red;')\
        .applymap(negativeValue, props='font-weight:bold;')
    return styled

In [16]:
print('Abi Eltern')
m = df_abiEltern.EO.DTE.mean()
print(m)
m = df_abiEltern.PE.DTE.mean()
print(m)
m = df_abiEltern.PP.DTE.mean()
print(m)
m = df_abiEltern.SA.DTE.mean()
print(m)

Abi Eltern
-2.2575890217081037e-05
-0.16238879566707667
-0.006422962887145986
-0.08118310988842983


In [17]:
s = showTable(df_abiEltern)
s